In [1]:
%cd ~/REVIVAL2
%load_ext autoreload
%autoreload 2
%load_ext blackcellmagic

/disk2/fli/REVIVAL2


In [8]:
import pandas as pd
from glob import glob
from scipy.stats import spearmanr

from REVIVAL.util import get_file_name

for dock_type in ["joint", "seperate"]:

    # Initialize an empty list to store results
    results = []

    # Loop through the CSV files and calculate Spearman correlation
    for lib in sorted(glob(f"/disk2/fli/REVIVAL2/zs/bonddist/chai/structure_{dock_type}/*.csv")):
        lib_name = get_file_name(lib)
        lib_df = pd.read_csv(lib)
        
        df_nan = lib_df[lib_df.isna().any(axis=1)]
        if len(df_nan) > 0:
            print(f"NaN values found in {lib_name}")
            print(df_nan)
        avg_c = [c for c in lib_df.columns if "agg" in c or "avg" in c]
        # std_c = [c for c in chai_df.columns if "std" in c]

        # Calculate Spearman correlation for each column in avg_c
        for c in avg_c:
            correlation, p_value = spearmanr(lib_df["fitness"].values, lib_df[c].values)
            # std_col = merged_df[c.replace("avg", "std")].values
            results.append({
                "Library": lib_name,
                "Score_Type": c,
                # "variability": std_col.mean(),
                "Spearman_Correlation": correlation,
                "P_Value": p_value
            })

    # Convert results into a DataFrame and display it as a table
    results_df = pd.DataFrame(results)
    display(results_df)
    display(results_df.loc[
        results_df.groupby("Library")["Spearman_Correlation"].idxmin()
    ])

,Library,Score_Type,Spearman_Correlation,P_Value
0,ParLQ,0:C-C_1_avg,-0.035191,0.437022
1,ParLQ,1:C-C_2_avg,0.010839,0.810859
2,PfTrpB-4bromo,0:C-C_avg,-0.156197,0.015219
3,PfTrpB-4bromo,1:GLU-NH_1_avg,-0.230684,0.000305
4,PfTrpB-4bromo,2:GLU-NH_2_avg,-0.232195,0.000277
5,PfTrpB-4cyano,0:C-C_avg,-0.037410,0.563299
6,PfTrpB-4cyano,1:GLU-NH_1_avg,0.236118,0.000217
7,PfTrpB-4cyano,2:GLU-NH_2_avg,0.230194,0.000314
8,PfTrpB-56chloro,0:C-C_avg,-0.210001,0.001039
9,PfTrpB-56chloro,1:GLU-NH_1_avg,0.024928,0.700207


,Library,Score_Type,Spearman_Correlation,P_Value
0,ParLQ,0:C-C_1_avg,-0.035191,0.437022
4,PfTrpB-4bromo,2:GLU-NH_2_avg,-0.232195,0.000277
5,PfTrpB-4cyano,0:C-C_avg,-0.037410,0.563299
8,PfTrpB-56chloro,0:C-C_avg,-0.210001,0.001039
11,PfTrpB-5bromo,0:C-C_avg,-0.179172,0.005277
14,PfTrpB-5chloro,0:C-C_avg,-0.096579,0.134910
19,PfTrpB-5cyano,2:GLU-NH_2_avg,-0.228690,0.000345
20,PfTrpB-5iodo,0:C-C_avg,-0.234247,0.000244
25,PfTrpB-6chloro,2:GLU-NH_2_avg,-0.212344,0.000909
26,PfTrpB-7bromo,0:C-C_avg,0.043707,0.723409


,Library,Score_Type,Spearman_Correlation,P_Value
0,ParLQ,0:C-C_1_avg,0.169302,0.000166
1,ParLQ,1:C-C_2_avg,0.134358,0.002882
2,Rma-CB,0:C-B_avg,-0.333174,0.000031
3,Rma-CSi,0:C-Si_avg,-0.104207,0.204423


,Library,Score_Type,Spearman_Correlation,P_Value
1,ParLQ,1:C-C_2_avg,0.134358,0.002882
2,Rma-CB,0:C-B_avg,-0.333174,0.000031
3,Rma-CSi,0:C-Si_avg,-0.104207,0.204423


In [9]:
import pandas as pd
from glob import glob
from scipy.stats import spearmanr

from REVIVAL.util import get_file_name

for dock_type in ["joint"]: # ["joint", "seperate"]:

    # Initialize an empty list to store results
    results = []

    # Loop through the CSV files and calculate Spearman correlation
    for lib in sorted(glob(f"/disk2/fli/REVIVAL2/zs/bonddist/af3/struct_{dock_type}/*.csv")):
        lib_name = get_file_name(lib)
        lib_df = pd.read_csv(lib)
        
        df_nan = lib_df[lib_df.isna().any(axis=1)]
        # if len(df_nan) > 0:
        #     print(f"NaN values found in {lib}")
        #     print(df_nan)
        avg_c = [c for c in lib_df.columns if "agg" in c or "avg" in c]
        # std_c = [c for c in chai_df.columns if "std" in c]

        # Calculate Spearman correlation for each column in avg_c
        for c in avg_c:
            lib_df_valid = lib_df.copy()
            valid_rows = lib_df_valid[["fitness", c]].dropna()
            correlation, p_value = spearmanr(valid_rows["fitness"].values, valid_rows[c].values)
            # correlation, p_value = spearmanr(lib_df["fitness"].values, lib_df[c].values)
            # std_col = merged_df[c.replace("avg", "std")].values
            results.append({
                "Library": lib_name,
                "Score_Type": c,
                # "variability": std_col.mean(),
                "Spearman_Correlation": correlation,
                "P_Value": p_value
            })

    # Convert results into a DataFrame and display it as a table
    results_df = pd.DataFrame(results)
    display(results_df)
    display(results_df.loc[
        results_df.groupby("Library")["Spearman_Correlation"].idxmin()
    ])

,Library,Score_Type,Spearman_Correlation,P_Value
0,ParLQ,0:C-C_1_agg,-0.292502,4.023011e-11
1,ParLQ,0:C-C_1_avg,-0.176340,8.699198e-05
2,ParLQ,1:C-C_2_agg,-0.282574,1.898597e-10
3,ParLQ,1:C-C_2_avg,-0.185062,3.760488e-05
4,PfTrpB-4bromo,0:C-C_agg,-0.305929,1.290775e-06
...,...,...,...,...
69,PfTrpB-7methyl,2:GLU-NH_2_avg,-0.221975,5.176284e-04
70,Rma-CB,0:C-B_agg,0.083652,3.402773e-01
71,Rma-CB,0:C-B_avg,0.077319,3.782115e-01
72,Rma-CSi,0:C-Si_agg,0.070772,4.200196e-01


,Library,Score_Type,Spearman_Correlation,P_Value
0,ParLQ,0:C-C_1_agg,-0.292502,4.023011e-11
8,PfTrpB-4bromo,2:GLU-NH_2_agg,-0.410826,3.135920e-11
15,PfTrpB-4cyano,2:GLU-NH_2_avg,-0.301838,1.813034e-06
20,PfTrpB-56chloro,2:GLU-NH_2_agg,-0.096161,1.366144e-01
22,PfTrpB-5bromo,0:C-C_agg,-0.260035,4.382111e-05
28,PfTrpB-5chloro,0:C-C_agg,-0.292876,3.748943e-06
35,PfTrpB-5cyano,0:C-C_avg,-0.126136,5.049012e-02
45,PfTrpB-5iodo,2:GLU-NH_2_avg,-0.111872,8.307153e-02
51,PfTrpB-6chloro,2:GLU-NH_2_avg,-0.234978,2.327427e-04
57,PfTrpB-7bromo,2:GLU-NH_2_avg,-0.210749,8.451003e-02
